# Conceptual Design of a Box-Wing Aircraft

In [110]:
from AeronauticConstants import AeronauticConstants as ac
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

import matplotlib as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The five independent equations are:

$$ x_R =  \frac{V_{stall,\ hyp.}^2}{2 T_0 x_{FR} g} \cdot Q_{MTO} \qquad \text{(Take-Off)} $$

$$ \frac{1}{2} \rho_{0} V_{stall,\ hyp.} ^ 2 S_w C_{L,\ hyp} = Q_{MTO} \qquad \text{(Lift-Off)} $$

$$ x_F = \frac{(V_{stall,\ hyp}^{Land})^2}{2 \bar{\bar a}} \qquad \text{(Braking)} $$

Note: $V_{stall,\ hyp}^{Land} \ne  V_{stall,\ hyp} $ as they are defined at two different weight conditions (\(Q_{MTO}\) and \(Q_{MTO} - Q_f\)).

$$ (V_{stall,\ hyp}^{Land})^2 = \frac{Q_{MTO} (1 - k)}{\frac{1}{2} \rho_0 S_w C_{L,\ hyp}} \cdot \frac{1}{\bar{\bar a}} \qquad (\text{Landing}) $$

$$ Q_{MTO} = Q_{OE} + Q_{f} + Q_{pay} \qquad (\text{Weights}) $$

Some of the parameters involved are actually from other assumptions or equations (which should increase their number to more than five); however, they are not included here, as these equations relate to the operative aspects of the aircraft and not to its setting (e.g., the autonomy depends on the engine SFC, which is fixed once the engine is fixed; however, the landing distance imposes an external condition on the aircraft setting, which is determined by the fact that the airports are of a certain size).

In [24]:
x_TO = (1300 + 1800) / 2 # [m] Balanced Take-Off Length: from full stop to virtual vertical obstalce at 35 ft (1300 m < x_TO < 1800 m)

z_CR = 20e3 * ac.FT_TO_M # [m] Cruise altitude

x_LD = (1000 + 1500) / 2 # [m] Balanced Landing Length: from V_land = 115 kts to full stop (1000 m < x_TO < 1500 m)

V_land = 115 * ac.KTS_TO_MPS # [m/s]

## $Q_{pay}$

From the Eq. (Weights), we start by evaluating the $Q_{pay}$ contribution, as it is given from the TLARs.

In [25]:
n_pax = 90 # [-] Number of passengers (80 < n_pax < 100)

Q_per_pax = 95 # [kg] mass per passenger

Q_pax = Q_per_pax * n_pax 

# ------ 

n_crew = 4 # [-] Number of company crew members

Q_crew = Q_per_pax * n_crew # [kg] 

# ------

Q_per_seat = 15 # [kg] # mass per seat 

Q_seats = Q_per_seat * n_pax # [kg]

# ------

Q_liq_0 = 200 # [kg]

Q_liq_add_per_pax = 1.9 # [kg]

Q_liq = Q_liq_0 + Q_liq_add_per_pax * n_pax # [kg]

# ------

V_per_lugg = 5 * (ac.FT_TO_M) ** 3 # [m^3]

V_lugg = V_per_lugg * n_pax # [m^3]

# ------

Q_pay = Q_pax + Q_crew + Q_seats + Q_liq

print(f"The total mass of payload to be carried is: Q_pay = {Q_pay} kg")

The total mass of payload to be carried is: Q_pay = 10651.0 kg



From CS 25.1 Applicability:

(a) These Certification Specifications are applicable to turbine powered Large Aeroplanes.

From wikipedia:

"This certification procedure applies to large, turbine-powered aircraft, with max take-off weight more than 5,700kg (CS 25.1)"

So we are in this Aircraft Category

## $ Q_{OE} $ 
We already determined $Q_{pay}$, the next step is to determine $Q_{EO}$. We will do so thanks to statistical formulae available in textbooks. In order for them to be valid, we have to make some adjustments and carefully check the founding hypothesis to ensure that they will work for our configuration (box-wing) as well.

In [ ]:
# Q_MTO = np.linspace(30e3, 50e3, 20) # [kg] Maximum Take-Off Mass
Q_MTO = 40e3

def ratio_Q_OE_estimate(Q_MTO, A = 0.97, C = -0.06):
    ratio_Q_OE = A * Q_MTO ** C
    return ratio_Q_OE

Q_OE = Q_MTO * ratio_Q_OE_estimate(Q_MTO)

## Autonomy range calculation

In [109]:
# Initial guess for Q_MTO
Q_MTO_start = 40e3  # [kg] initial guess
tolerance = 100  # [kg] Convergence criterion 
max_iter = 100
iteration = 0

Q_f_non_usable = 0.003
ratio_Q_f_TO = 0.97
ratio_Q_f_CL = 0.985
ratio_Q_f_LG = 0.995
E_CR = 17
mach_CR = 0.8

# Load SFC data once
data_sfc = pd.read_csv("high_bpr_sfc_vs_mach.csv")
mach = data_sfc["mach"]
sfc = data_sfc["sfc"]
sfc_interp = interp1d(mach, sfc)
mach_dense = np.linspace(mach.min(), mach.max(), 100)
sfc_dense = sfc_interp(mach_dense)
idx_SFC_CR = np.argmin(np.abs(mach_CR - mach_dense))
SFC_CR = sfc_dense[idx_SFC_CR] * (1 / ac.H_TO_S)  # [1/s]

# Cruise speed
T_CR = ac.T_isa(z_CR)
V_CR = mach_CR * ac.c_sound(T_CR)
x_range = 1000 * ac.NM_TO_KM * 1000  # [m] Required range

# Iterative loop
Q_MTO = Q_MTO_start
while iteration < max_iter:
    iteration += 1
    
    # Estimate OE mass
    Q_OE = Q_MTO * ratio_Q_OE_estimate(Q_MTO)
    
    # Fuel fraction for cruise
    ratio_Q_f_CR = np.exp(-SFC_CR * x_range / (0.866 * E_CR * V_CR))
    ratio_Q_f_mission = ratio_Q_f_TO * ratio_Q_f_CL * ratio_Q_f_CR * ratio_Q_f_LG 
    ratio_Q_f_mission = (1 + Q_f_non_usable) * (1 - ratio_Q_f_mission) 

    # Fuel mass
    Q_f = ratio_Q_f_mission * Q_MTO
    
    # Total aircraft mass for next iteration
    Q_MTO_new = Q_f + Q_pay + Q_OE

    # Check convergence
    if abs(Q_MTO_new - Q_MTO) < tolerance:
        print(f"Converged after {iteration} iterations.")
        break
    
    # Update Q_MTO for next iteration
    Q_MTO = Q_MTO_new

# Results
print(f"Starting Q_MTO = {Q_MTO_start:.2e} kg")
print(f"Starting Q_pay = {Q_pay:.2e} kg")
print(f"Final Q_f = {Q_f:.2e} kg")
print(f"Final Q_OE = {Q_OE:.2e} kg")
print(f"Total aircraft mass = {Q_f + Q_pay + Q_OE:.2e} kg")

Converged after 9 iterations.
Starting Q_MTO = 4.00e+04 kg
Starting Q_pay = 1.07e+04 kg
Final Q_f = 3.90e+03 kg
Final Q_OE = 1.60e+04 kg
Total aircraft mass = 3.05e+04 kg


# Components Weight estimation

## Wings

Empirical formula from [Raymer](https://aircraftdesign.com/wp-content/uploads/2024/04/WtsMethods_used_in_ADACA-CargoTransport.pdf)

In [145]:
def Q_wing_estimate(Q_MTO, N_load, S_wing, AR, thick_airfoil_r, TR, sweep_25, S_ctrl):
    
    """
    Q_MTO: Maximum Take off Weight [kg]
    N_load: Ultimate load factor [-]
    S_wing: Gross wing area [m^2]
    AR: Wing aspect ratio [-]
    thick_airfoil_r: Wing root thickness divided by wing root chord
    TR: Wing taper ratio [-]
    sweep_25: Wing sweep angle at 25% chord [deg]
    S_ctrl: Area of wing mounted control surfaces [m^2]
    """

    Q_wing = 0.0051 * (Q_MTO * ac.KG_TO_LB * N_load) ** (0.557) * (S_wing * ac.M_TO_FT ** 2) ** (0.649) * AR ** (0.5) * thick_airfoil_r ** (- 0.4) * \
    (1 + TR) ** (0.1) / np.cos(np.deg2rad(sweep_25)) * (S_ctrl * ac.M_TO_FT ** 2) ** (0.100)
    return Q_wing

In [146]:
data = [
    ("AC-1DH", 26000, 4.97, 912, 10.02, 0.175, 1.48, 0, 285, 3311),
    ("C-123B", 54000, 4.50, 1223, 9.89, 0.167, 1.53, 0, 211, 5906),
    ("C-124C",185000, 3.75, 2506,12.10, 0.190, 1.24, 0, 654,18135),
    ("C-130B",135000, 4.16, 1745,10.08, 0.180, 1.52, 0, 452,11145),
    ("C-131B", 53200, 3.15,  920,12.06, 0.200, 1.33, 0, 231, 5072),
    ("C-133A",275000, 3.75, 2673,12.07, 0.170, 1.23, -6, 640,27404),
    ("C-135A",270000, 3.75, 2433, 7.04, 0.167, 1.33, 25, 615,25109),
    ("C-135B",274000, 3.75, 2433, 7.04, 0.167, 1.33, 35, 615,25250),
    ("C-141A",316100, 3.75, 3228, 8.00, 0.130, 1.33, 25,1011,34475),
    ("C-5A"  ,728000, 3.75, 6200, 8.00, 0.130, 1.34, 25,2343,85750),
    ("XC-142A",37474, 4.50,  534, 8.53, 0.180, 1.61, 4, 397,2633)
]

for name, W_dg, N_z, S_w, AR, t_c_r, TR, sweep, S_wcs, W_true in data:
    W_est = Q_wing_estimate(W_dg * ac.LB_TO_KG, N_z, S_w * ac.FT_TO_M ** 2, AR, t_c_r, TR, sweep, S_wcs * ac.FT_TO_M ** 2)
    print(f"{name:8s}  Estimated: {W_est:8.0f}  Actual: {W_true:8.0f}  Error: {100*(W_est-W_true)/W_true:6.1f}%")


AC-1DH    Estimated:     3662  Actual:     3311  Error:   10.6%
C-123B    Estimated:     6199  Actual:     5906  Error:    5.0%
C-124C    Estimated:    20584  Actual:    18135  Error:   13.5%
C-130B    Estimated:    13158  Actual:    11145  Error:   18.1%
C-131B    Estimated:     4308  Actual:     5072  Error:  -15.1%
C-133A    Estimated:    28031  Actual:    27404  Error:    2.3%
C-135A    Estimated:    22041  Actual:    25109  Error:  -12.2%
C-135B    Estimated:    24587  Actual:    25250  Error:   -2.6%
C-141A    Estimated:    35802  Actual:    34475  Error:    3.8%
C-5A      Estimated:    94703  Actual:    85750  Error:   10.4%
XC-142A   Estimated:     2852  Actual:     2633  Error:    8.3%
